In [2]:
import networkx as nx

def load_facebook_graph(path="facebook"):
    G = nx.Graph()
    with open(path, "r") as f:
        # Đọc dòng đầu: có hai số (số nút, số cạnh×2)
        header = f.readline().strip().split()
        # Nếu đúng là hai số, ta bỏ qua; còn nếu không phải thì dùng nó làm cạnh
        if len(header) == 2 and header[0].isdigit() and header[1].isdigit():
            n_nodes, twice_edges = map(int, header)
        else:
            # coi là dòng cạnh đầu
            u, v = map(int, header)
            G.add_edge(u, v)

        # Đọc phần còn lại, mỗi dòng một cạnh u v
        for line in f:
            parts = line.strip().split()
            if len(parts) < 2:
                continue
            u, v = map(int, parts[:2])
            G.add_edge(u, v)
    return G

# Ví dụ chạy ngay
G = load_facebook_graph("facebook")
print(f"Số nút: {G.number_of_nodes():,}  |  Số cạnh: {G.number_of_edges():,}")



Số nút: 4,039  |  Số cạnh: 88,234


In [33]:
import sys
!{sys.executable} -m pip install networkx torch torchvision torchaudio


  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.8/212.5 MB 8.3 MB/s eta 0:00:26
   ---------------------------------------- 1.8/212.5 MB 8.4 MB/s eta 0:00:26
    --------------------------------------- 3.7/212.5 MB 7.0 MB/s eta 0:00:30
   - -------------------------------------- 5.5/212.5 MB 7.8 MB/s eta 0:00:27
   - -------------------------------------- 7.3/212.5 MB 8.1 MB/s eta 0:00:26
   - -------------------------------------- 9.2/212.5 MB 8.3 MB/s eta 0:00:25
   -- ------------------------------------- 11.3/212.5 MB 8.5 MB/s eta 0:00:24
   -- ------------------------------------- 13.1/212.5 MB 8.7 MB/s eta 0:00:24
   -- ------------------------------------- 15.2/212.5 MB 8.8 MB/s eta 0:00:23
   --- ------------------------------------ 17.0/212.5 MB 8.9 MB/s eta 0:00:22
   --- ------------------------------------ 19.1/212.5 MB 9.0 MB/s eta 0:00:22
   -

In [3]:
import networkx as nx
import random
from typing import Dict, List, Set, Tuple
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# --- Multi-Topic Independent Cascade Simulation ---
class MultiTopicIC:
    def __init__(self, G: nx.DiGraph, k: int,
                 prob: Dict[int, Dict[Tuple[int, int], float]]):
        self.G = G
        self.k = k
        self.prob = prob

    def simulate(self, seeds: List[Set[int]]) -> List[Set[int]]:
        activated = [set(S) for S in seeds]
        frontier = [set(S) for S in seeds]
        while any(frontier):
            new_frontier = [set() for _ in range(self.k)]
            for i in range(self.k):
                for u in frontier[i]:
                    for v in self.G.successors(u):
                        if v not in activated[i] and random.random() <= self.prob[i].get((u, v), 0.0):
                            new_frontier[i].add(v)
                            activated[i].add(v)
            frontier = new_frontier
        return activated

    def expected_spread(self, seeds: List[Set[int]], runs: int = 100) -> List[float]:
        total = [0.0] * self.k
        for _ in range(runs):
            act = self.simulate(seeds)
            for i in range(self.k):
                total[i] += len(act[i])
        return [t / runs for t in total]

# --- Probability Generator ---
def generate_probabilities(G: nx.DiGraph, k: int, sigma: float = 0.1) -> Dict[int, Dict[Tuple[int, int], float]]:
    indeg = {v: G.in_degree(v) if G.in_degree(v) > 0 else 1 for v in G.nodes()}
    probs: Dict[int, Dict[Tuple[int, int], float]] = {i: {} for i in range(k)}
    for i in range(k):
        for u, v in G.edges():
            base_p = 1.0 / indeg[v]
            noise = random.gauss(0, sigma)
            p = min(max(base_p + noise, 0.0), 1.0)
            probs[i][(u, v)] = p
    return probs

# --- MLP Model for Influence Prediction ---
class InfluenceMLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dims: List[int] = [64, 32]):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.ReLU())
            prev = h
        layers.append(nn.Linear(prev, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x).squeeze(-1)

# --- Feature Extraction ---
def extract_features(seed_set: Set[int], degree_centrality: Dict[int, float], topic: int) -> torch.Tensor:
    sum_deg = sum(degree_centrality.get(v, 0.0) for v in seed_set)
    size = len(seed_set)
    return torch.tensor([sum_deg, size, float(topic)], dtype=torch.float32)

# --- Greedy Search for Optimal Seed Sets ---
def search_optimal_seeds(model: InfluenceMLP,
                         k: int,
                         G: nx.DiGraph,
                         degree_centrality: Dict[int, float],
                         budget: List[int]) -> List[Set[int]]:
    seeds = [set() for _ in range(k)]
    nodes = set(G.nodes())
    for i in range(k):
        for _ in range(budget[i]):
            best_node, best_gain = None, -float('inf')
            current_feat = extract_features(seeds[i], degree_centrality, i)
            current_pred = model(current_feat.unsqueeze(0))[0].item()
            for v in nodes - seeds[i]:
                feat = extract_features(seeds[i] | {v}, degree_centrality, i)
                pred = model(feat.unsqueeze(0))[0].item()
                gain = pred - current_pred
                if gain > best_gain:
                    best_gain, best_node = gain, v
            if best_node is None:
                break
            seeds[i].add(best_node)
    return seeds

# --- Main Workflow ---
if __name__ == "__main__":
    # Load graph
    path = "facebook"
    G_undirected = nx.read_edgelist(path, nodetype=int)
    G = G_undirected.to_directed()

    # Generate topic-specific probabilities
    k = 2
    sigma_noise = 0.1
    prob = generate_probabilities(G, k, sigma_noise)
    ic_model = MultiTopicIC(G, k, prob)

    # Precompute degree centrality
    degree_centrality = nx.degree_centrality(G)

    # Prepare training data by sampling random seed sets
    budget = [2] * k  # number of seeds per topic
    num_samples = 500
    X_list, y_list = [], []
    for topic in range(k):
        for _ in range(num_samples):
            S = set(random.sample(list(G.nodes()), budget[topic]))
            feat = extract_features(S, degree_centrality, topic)
            spread = ic_model.expected_spread([S if t == topic else set() for t in range(k)], runs=50)[topic]
            X_list.append(feat)
            y_list.append(spread)

    # Build DataLoader
    X = torch.stack(X_list)
    y = torch.tensor(y_list, dtype=torch.float32)
    dataset = TensorDataset(X, y)
    loader = DataLoader(dataset, batch_size=64, shuffle=True)

    # Initialize and train MLP
    model = InfluenceMLP(input_dim=3)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    epochs = 20
    for epoch in range(epochs):
        total_loss = 0.0
        for xb, yb in loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * xb.size(0)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataset):.4f}")

    # Find optimal seed sets using trained MLP
    optimal_seeds = search_optimal_seeds(model, k, G, degree_centrality, budget)
    print("Optimal seed sets:", optimal_seeds)

    # Compute predicted spread
    spreads = [model(extract_features(optimal_seeds[i], degree_centrality, i).unsqueeze(0))[0].item() for i in range(k)]
    total_spread = sum(spreads)
    print("Predicted spread per topic:", spreads)
    print("Total predicted spread:", total_spread)


Epoch 1/20, Loss: 4411413.3040
Epoch 2/20, Loss: 4408861.8200
Epoch 3/20, Loss: 4405109.6320
Epoch 4/20, Loss: 4399414.8880
Epoch 5/20, Loss: 4390926.0400
Epoch 6/20, Loss: 4378804.8320
Epoch 7/20, Loss: 4361847.2480
Epoch 8/20, Loss: 4338994.4520
Epoch 9/20, Loss: 4309517.7100
Epoch 10/20, Loss: 4272101.8200
Epoch 11/20, Loss: 4226191.6960
Epoch 12/20, Loss: 4170607.9520
Epoch 13/20, Loss: 4104160.5000
Epoch 14/20, Loss: 4024498.4920
Epoch 15/20, Loss: 3932422.5440
Epoch 16/20, Loss: 3827764.0000
Epoch 17/20, Loss: 3709595.5680
Epoch 18/20, Loss: 3579219.7760
Epoch 19/20, Loss: 3437007.2480
Epoch 20/20, Loss: 3282201.5960
Optimal seed sets: [{107, 1684}, {107, 1684}]
Predicted spread per topic: [355.09832763671875, 435.040771484375]
Total predicted spread: 790.1390991210938


In [4]:
import networkx as nx
import random
from typing import Dict, List, Set, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# --- 1. Multi-Topic Independent Cascade with Gaussian Noise ---
class MultiTopicIC:
    def __init__(self,
                 G: nx.DiGraph,
                 k: int,
                 base_prob: Dict[int, Dict[Tuple[int, int], float]],
                 sigma: float = 0.1):
        self.G = G
        self.k = k
        self.base_prob = base_prob
        self.sigma = sigma

    def _noisy_prob(self, i: int, u: int, v: int) -> float:
        p0 = self.base_prob[i].get((u, v), 0.0)
        return float(min(max(p0 + random.gauss(0, self.sigma), 0.0), 1.0))

    def simulate(self, seeds: List[Set[int]]) -> List[Set[int]]:
        # giữ non-overlap: nếu node active topic A, không active B
        activated = [set(S) for S in seeds]
        frontier = [set(S) for S in seeds]

        # track globally active to enforce competition
        global_active: Dict[int, int] = {v: -1 for v in self.G.nodes()}
        for i, S in enumerate(seeds):
            for v in S:
                global_active[v] = i

        while any(frontier):
            new_frontier = [set() for _ in range(self.k)]
            for i in range(self.k):
                for u in frontier[i]:
                    for v in self.G.successors(u):
                        if global_active[v] == -1 and random.random() <= self._noisy_prob(i, u, v):
                            global_active[v] = i
                            new_frontier[i].add(v)
                            activated[i].add(v)
            frontier = new_frontier
        return activated

    def expected_spread(self, seeds: List[Set[int]], runs: int = 100) -> List[float]:
        cum = [0.0] * self.k
        for _ in range(runs):
            act = self.simulate(seeds)
            for i in range(self.k):
                cum[i] += len(act[i])
        return [x / runs for x in cum]


# --- 2. Generate base probabilities per topic ---
def generate_base_probs(G: nx.DiGraph, k: int) -> Dict[int, Dict[Tuple[int, int], float]]:
    # sử dụng 1 / indegree làm base
    indeg = {v: max(G.in_degree(v), 1) for v in G.nodes()}
    probs: Dict[int, Dict[Tuple[int, int], float]] = {i: {} for i in range(k)}
    for i in range(k):
        for u, v in G.edges():
            probs[i][(u, v)] = 1.0 / indeg[v]
    return probs


# --- 3. MLP for influence prediction ---
class InfluenceMLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dims: List[int] = [64, 32]):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers += [nn.Linear(prev, h), nn.ReLU()]
            prev = h
        layers.append(nn.Linear(prev, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x).squeeze(-1)


# --- 4. Feature extraction per (seed_set, topic) ---
def extract_feature(seed_set: Set[int],
                    degree_centrality: Dict[int, float],
                    topic: int) -> torch.Tensor:
    # ví dụ: [|S|, sum_deg, topic_id]
    return torch.tensor([
        float(len(seed_set)),
        sum(degree_centrality[v] for v in seed_set),
        float(topic)
    ], dtype=torch.float32)


# --- 5. Greedy search ensure non-overlap & per-topic budget ---
def search_optimal_seeds(
    model: InfluenceMLP,
    G: nx.DiGraph,
    degree_centrality: Dict[int, float],
    k: int,
    budget_per_topic: List[int]
) -> List[Set[int]]:
    chosen: List[Set[int]] = [set() for _ in range(k)]
    available = set(G.nodes())

    for i in range(k):
        for _ in range(budget_per_topic[i]):
            best_node, best_gain = None, -1e9
            base_feat = extract_feature(chosen[i], degree_centrality, i).unsqueeze(0)
            base_pred = model(base_feat).item()
            for v in available - chosen[i]:
                feat = extract_feature(chosen[i] | {v}, degree_centrality, i).unsqueeze(0)
                gain = model(feat).item() - base_pred
                if gain > best_gain:
                    best_gain, best_node = gain, v
            if best_node is None:
                break
            chosen[i].add(best_node)
            available.remove(best_node)  # enforce non-overlap
    return chosen


# --- 6. Main training & search flow ---
if __name__ == "__main__":
    path = "facebook"
    G_undirected = nx.read_edgelist(path, nodetype=int)
    G = G_undirected.to_directed()
    # 2. prepare IC model
    k = 3
    base_prob = generate_base_probs(G, k)
    ic = MultiTopicIC(G, k, base_prob, sigma=0.1)

    # 3. degree centrality
    deg_c = nx.degree_centrality(G)

    # 4. generate training samples
    budget = [5] * k
    samples = []
    targets = []
    for topic in range(k):
        for _ in range(300):
            S = set(random.sample(list(G.nodes()), budget[topic]))
            # chỉ tính spread cho đúng topic, các topic khác empty
            spreads = ic.expected_spread([S if t == topic else set() for t in range(k)], runs=30)
            samples.append(extract_feature(S, deg_c, topic))
            targets.append(spreads[topic])

    X = torch.stack(samples)       # [N, 3]
    y = torch.tensor(targets)      # [N]
    ds = TensorDataset(X, y)
    loader = DataLoader(ds, batch_size=64, shuffle=True)

    # 5. init & train MLP
    model = InfluenceMLP(input_dim=3)
    opt = optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()
    for epoch in range(25):
        epoch_loss = 0
        for xb, yb in loader:
            opt.zero_grad()
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            epoch_loss += loss.item() * xb.size(0)
        print(f"Epoch {epoch+1} Loss: {epoch_loss/len(ds):.4f}")

    # 6. search optimal seeds
    optimal = search_optimal_seeds(model, G, deg_c, k, budget)
    print("Optimal seed sets:", optimal)
    # 7. estimate final spread
    final = ic.expected_spread(optimal, runs=100)
    print("Final expected spread per topic:", final)


Epoch 1 Loss: 6486516.7000
Epoch 2 Loss: 6478378.0356
Epoch 3 Loss: 6466759.3778
Epoch 4 Loss: 6449385.6711
Epoch 5 Loss: 6423914.3044
Epoch 6 Loss: 6387855.6933
Epoch 7 Loss: 6338559.2889
Epoch 8 Loss: 6273408.6000
Epoch 9 Loss: 6189761.4089
Epoch 10 Loss: 6085232.7978
Epoch 11 Loss: 5957884.2756
Epoch 12 Loss: 5805949.9178
Epoch 13 Loss: 5627688.9378
Epoch 14 Loss: 5423278.3978
Epoch 15 Loss: 5192461.2889
Epoch 16 Loss: 4935960.1333
Epoch 17 Loss: 4656010.9867
Epoch 18 Loss: 4353101.7178
Epoch 19 Loss: 4030545.3278
Epoch 20 Loss: 3691706.8778
Epoch 21 Loss: 3344061.7822
Epoch 22 Loss: 2991862.0200
Epoch 23 Loss: 2641128.8200
Epoch 24 Loss: 2296772.7489
Epoch 25 Loss: 1966447.0989
Optimal seed sets: [{0, 107, 3437, 1684, 1912}, {1888, 1800, 2347, 2543, 1663}, {1730, 483, 1352, 2266, 348}]
Final expected spread per topic: [2105.38, 355.72, 352.53]
